In [13]:
import tensorflow as tf
import keras

In [14]:
X = tf.range(10)
dataset = tf.data.Dataset.from_tensor_slices(X)
dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [15]:
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


2025-08-23 07:08:37.995158: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [16]:
dataset = dataset.repeat(3).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9], shape=(2,), dtype=int32)


In [17]:
dataset = dataset.map(lambda x : x ** 2)
dataset

<_MapDataset element_spec=TensorSpec(shape=(None,), dtype=tf.int32, name=None)>

In [18]:
for item in dataset:
    print(item)

tf.Tensor([ 0  1  4  9 16 25 36], shape=(7,), dtype=int32)
tf.Tensor([49 64 81  0  1  4  9], shape=(7,), dtype=int32)
tf.Tensor([16 25 36 49 64 81  0], shape=(7,), dtype=int32)
tf.Tensor([ 1  4  9 16 25 36 49], shape=(7,), dtype=int32)
tf.Tensor([64 81], shape=(2,), dtype=int32)


In [19]:
def some_func(X):
    y = 10
    return X + y

In [20]:
dataset = dataset.map(some_func)
for item in dataset:
    print(item)

tf.Tensor([10 11 14 19 26 35 46], shape=(7,), dtype=int32)
tf.Tensor([59 74 91 10 11 14 19], shape=(7,), dtype=int32)
tf.Tensor([26 35 46 59 74 91 10], shape=(7,), dtype=int32)
tf.Tensor([11 14 19 26 35 46 59], shape=(7,), dtype=int32)
tf.Tensor([74 91], shape=(2,), dtype=int32)


In [24]:
dataset = dataset.filter(lambda x : tf.reduce_sum(x) > 200)
for item in dataset:
    print(item)

tf.Tensor([59 74 91 10 11 14 19], shape=(7,), dtype=int32)
tf.Tensor([26 35 46 59 74 91 10], shape=(7,), dtype=int32)
tf.Tensor([11 14 19 26 35 46 59], shape=(7,), dtype=int32)


In [38]:
dataset = tf.data.Dataset.range(10).repeat(2)
dataset = dataset.shuffle(buffer_size=4,seed = 42,reshuffle_each_iteration=False).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 1 3 4 5 2 6], shape=(7,), dtype=int64)
tf.Tensor([0 9 1 7 2 3 8], shape=(7,), dtype=int64)
tf.Tensor([5 8 7 9 4 6], shape=(6,), dtype=int64)


In [41]:
for item in dataset.repeat(2):
    print(item)

tf.Tensor([0 1 3 4 5 2 6], shape=(7,), dtype=int64)
tf.Tensor([0 9 1 7 2 3 8], shape=(7,), dtype=int64)
tf.Tensor([5 8 7 9 4 6], shape=(6,), dtype=int64)
tf.Tensor([0 1 3 4 5 2 6], shape=(7,), dtype=int64)
tf.Tensor([0 9 1 7 2 3 8], shape=(7,), dtype=int64)
tf.Tensor([5 8 7 9 4 6], shape=(6,), dtype=int64)


In [44]:
with tf.io.TFRecordWriter('dummy.tfrecord') as f:
    f.write(b'This is first record')
    f.write(b'This is second record')

In [ ]:
data = tf.data.TFRecordDataset('dummy.tfrecord',num_parallel_reads=2)
for item in data:
    print(item)

tf.Tensor(b'This is first record', shape=(), dtype=string)
tf.Tensor(b'This is second record', shape=(), dtype=string)


2025-08-23 09:28:43.881805: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [48]:
options = tf.io.TFRecordOptions(compression_type="GZIP")
with tf.io.TFRecordWriter("my_compressed.tfrecord", options) as f:
    f.write(b"Compress, compress, compress!")

In [49]:
for item in dataset:
    print(item)

tf.Tensor([0 1 3 4 5 2 6], shape=(7,), dtype=int64)
tf.Tensor([0 9 1 7 2 3 8], shape=(7,), dtype=int64)
tf.Tensor([5 8 7 9 4 6], shape=(6,), dtype=int64)


In [53]:
from sample_pb2 import Person

person = Person(name = 'Al' , id = 123 , email = ['a@b.com'])
person

name: "Al"
id: 123
email: "a@b.com"

In [55]:
person.name

'Al'

In [56]:
serialized = person.SerializeToString()
serialized

b'\n\x02Al\x10{\x1a\x07a@b.com'

In [58]:
person2 = Person()
person2

In [61]:
person2.ParseFromString(serialized)

15

In [62]:
person == person2

True

In [66]:
# person_tf = tf.io.decode_proto(
#     bytes=serialized,
#     message_type="Person",
#     field_names=["name", "id", "email"],
#     output_types=[tf.string, tf.int32, tf.string],
#     descriptor_source="sample.desc")

# person_tf.values

In [68]:
from tensorflow.train import BytesList, FloatList, Int64List
from tensorflow.train import Feature, Features, Example

person_example = Example(
    features=Features(
        feature={
            "name": Feature(bytes_list=BytesList(value=[b"Alice"])),
            "id": Feature(int64_list=Int64List(value=[123])),
            "emails": Feature(bytes_list=BytesList(value=[b"a@b.com",
                                                          b"c@d.com"]))
        }))

In [70]:
with tf.io.TFRecordWriter('my_contacts.tfrecord') as f:
    for _ in range(5):
        print(person_example.SerializeToString())
        f.write(person_example.SerializeToString())

b'\n@\n\x1e\n\x06emails\x12\x14\n\x12\n\x07a@b.com\n\x07c@d.com\n\x11\n\x04name\x12\t\n\x07\n\x05Alice\n\x0b\n\x02id\x12\x05\x1a\x03\n\x01{'
b'\n@\n\x1e\n\x06emails\x12\x14\n\x12\n\x07a@b.com\n\x07c@d.com\n\x11\n\x04name\x12\t\n\x07\n\x05Alice\n\x0b\n\x02id\x12\x05\x1a\x03\n\x01{'
b'\n@\n\x1e\n\x06emails\x12\x14\n\x12\n\x07a@b.com\n\x07c@d.com\n\x11\n\x04name\x12\t\n\x07\n\x05Alice\n\x0b\n\x02id\x12\x05\x1a\x03\n\x01{'
b'\n@\n\x1e\n\x06emails\x12\x14\n\x12\n\x07a@b.com\n\x07c@d.com\n\x11\n\x04name\x12\t\n\x07\n\x05Alice\n\x0b\n\x02id\x12\x05\x1a\x03\n\x01{'
b'\n@\n\x1e\n\x06emails\x12\x14\n\x12\n\x07a@b.com\n\x07c@d.com\n\x11\n\x04name\x12\t\n\x07\n\x05Alice\n\x0b\n\x02id\x12\x05\x1a\x03\n\x01{'


In [71]:
feature_description = {
    'name' : tf.io.FixedLenFeature([],tf.string,default_value=''),
    'id' : tf.io.FixedLenFeature([],tf.int64,default_value=0),
    'emails' : tf.io.VarLenFeature(tf.string)
}

def parse(serialized_example):
    return tf.io.parse_single_example(serialized_example,feature_description)

dataset = tf.data.TFRecordDataset(['my_contacts.tfrecord']).map(parse)

for parsed_example in dataset:
    print(parsed_example)

{'emails': SparseTensor(indices=tf.Tensor(
[[0]
 [1]], shape=(2, 1), dtype=int64), values=tf.Tensor([b'a@b.com' b'c@d.com'], shape=(2,), dtype=string), dense_shape=tf.Tensor([2], shape=(1,), dtype=int64)), 'id': <tf.Tensor: shape=(), dtype=int64, numpy=123>, 'name': <tf.Tensor: shape=(), dtype=string, numpy=b'Alice'>}
{'emails': SparseTensor(indices=tf.Tensor(
[[0]
 [1]], shape=(2, 1), dtype=int64), values=tf.Tensor([b'a@b.com' b'c@d.com'], shape=(2,), dtype=string), dense_shape=tf.Tensor([2], shape=(1,), dtype=int64)), 'id': <tf.Tensor: shape=(), dtype=int64, numpy=123>, 'name': <tf.Tensor: shape=(), dtype=string, numpy=b'Alice'>}
{'emails': SparseTensor(indices=tf.Tensor(
[[0]
 [1]], shape=(2, 1), dtype=int64), values=tf.Tensor([b'a@b.com' b'c@d.com'], shape=(2,), dtype=string), dense_shape=tf.Tensor([2], shape=(1,), dtype=int64)), 'id': <tf.Tensor: shape=(), dtype=int64, numpy=123>, 'name': <tf.Tensor: shape=(), dtype=string, numpy=b'Alice'>}
{'emails': SparseTensor(indices=tf.Tenso